# Clean Metadata
This notebook processes raw metadata provided by GISAID into the format required by Augur. Furthermore, it extracts additional information into unique variables.

## Setup

### Imports

In [1]:
import numpy as np
import pandas as pd

### Files

#### Inputs

In [2]:
input_gisaid = 'data/raw/southamerica_metadata.xls'

#### Outputs



In [3]:
export_augur = 'data/clean/southamerica_metadata_clean.tsv'

### Parameters

## Load raw data from GISAID

In [4]:
meta_raw = pd.DataFrame(pd.read_excel(input_gisaid))
meta_raw.head()

,Isolate_Id,PB2 Segment_Id,PB1 Segment_Id,PA Segment_Id,HA Segment_Id,NP Segment_Id,NA Segment_Id,MP Segment_Id,NS Segment_Id,HE Segment_Id,...,PB2 INSDC_Upload,PB1 INSDC_Upload,PA INSDC_Upload,HA INSDC_Upload,NP INSDC_Upload,NA INSDC_Upload,MP INSDC_Upload,NS INSDC_Upload,HE INSDC_Upload,P3 INSDC_Upload
0,EPI_ISL_18698459,EPI2872452|A/Goose/Argentina/389-1/2023_PB2,EPI2872453|A/Goose/Argentina/389-1/2023_PB1,EPI2872451|A/Goose/Argentina/389-1/2023_PA,EPI2872455|A/Goose/Argentina/389-1/2023_HA,EPI2872448|A/Goose/Argentina/389-1/2023_NP,EPI2872454|A/Goose/Argentina/389-1/2023_NA,EPI2872450|A/Goose/Argentina/389-1/2023_MP,EPI2872449|A/Goose/Argentina/389-1/2023_NS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EPI_ISL_17950994,NaN,NaN,NaN,EPI2610769|A/goose/Argentina/SENASA-140223/202...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EPI_ISL_17885869,EPI2609130|A/goose/Araucania/239189-1/2023_PB2,EPI2609128|A/goose/Araucania/239189-1/2023_PB1,EPI2644589|A/goose/Araucania/239189-1/2023_PA,EPI2609126|A/goose/Araucania/239189-1/2023_HA,EPI2609127|A/goose/Araucania/239189-1/2023_NP,EPI2637161|A/goose/Araucania/239189-1/2023_NA,EPI2609131|A/goose/Araucania/239189-1/2023_MP,EPI2609129|A/goose/Araucania/239189-1/2023_NS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,EPI_ISL_17527083,NaN,NaN,NaN,EPI2528164|A/goose/Argentina/140223/2023,NaN,EPI2528165|A/goose/Argentina/140223/2023,NaN,NaN,NaN,...,NaN,NaN,NaN,OQ832505,NaN,OQ832506,NaN,NaN,NaN,NaN
4,EPI_ISL_18777130,EPI2921506|A/gull/Maule/SJCEIRR-2343291/2023,NaN,EPI2921502|A/gull/Maule/SJCEIRR-2343291/2023,EPI2921503|A/gull/Maule/SJCEIRR-2343291/2023,EPI2921504|A/gull/Maule/SJCEIRR-2343291/2023,EPI2921501|A/gull/Maule/SJCEIRR-2343291/2023,EPI2921507|A/gull/Maule/SJCEIRR-2343291/2023,EPI2921505|A/gull/Maule/SJCEIRR-2343291/2023,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Keep only the columns we are interested in.

In [5]:
retained_cols = [
    'Isolate_Name',
    'Collection_Date',
    'Isolate_Id',
    'Location',
    'Host',
    'Submitting_Lab',
    'Originating_Lab']

meta_working = meta_raw[retained_cols]

Standardize column name formatting:
- all lowercase
- underscore for word separation (already true)

In [6]:
meta_working.columns = [c.lower() for c in meta_working.columns]

Provide [required columns](https://docs.nextstrain.org/projects/ncov/en/wdl-optionals/analysis/data-prep.html#required-metadata) for Nextstrain.

In [7]:
meta_working = meta_working.rename(
    columns = {
        'isolate_name': 'strain',
        'collection_date': 'date'
    }
)
meta_working['virus'] = 'avian_flu'

## Parse metadata

### Parse country

In [8]:
split_columns = meta_working['location'].str.split(' / ', expand=True)
meta_working['region'] = split_columns[0]
meta_working['country'] = split_columns[1]

### Parse host

In [9]:
# Rename raw host column:
meta_working = meta_working.rename(
    columns = {
        'host': 'host_raw',
    }
)

# Empirical lists of avian and human values
hosts_avian = ['Chicken', 'Other avian', 'Avian',
               'Wild bird', 'Turkey', 'Duck',
               'Gull', 'Gallus gallus domesticus',
               'Goose', 'Penguin', 'Swan',
               'Anas platyrhynchos var. domesticus',
               'Anas cyanoptera', 'Rynchops niger',
               'Cormorant', 'Calidris alba',
               'Larosterna inca']
hosts_mammal = ['Other mammals', 'Feline']

def label_host(val):
    if val in hosts_avian:
        return 'Avian'
    elif val in hosts_mammal:
        return 'Mammal'
    elif val == 'Human':
        return 'Human'
    else:
        return np.nan

meta_working['host'] = (
    meta_working['host_raw']
    .apply(label_host))

## Clean dataframe
Clean up values.

Clean strain names

In [10]:
meta_working['strain'][100]

'A/sea lion/Brazil/KU-3584/2023'

In [11]:
meta_working['strain'] = (
    meta_working['strain']
    .str.strip()
)
meta_working['strain'] = (
    meta_working['strain']
    .str.replace(' ', '_')
)

In [12]:
meta_working['strain'][100]

'A/sea_lion/Brazil/KU-3584/2023'

### Clean up countries

In [13]:
meta_working['country'].value_counts(dropna=False)

country
Chile                                101
Peru                                  64
Argentina                             55
Uruguay                               12
Ecuador                                8
Brazil                                 7
Colombia                               6
Venezuela, Bolivarian Republic of      5
Name: count, dtype: int64

In [14]:
meta_working['country'] = (
    meta_working['country']
    .replace(
        {'Venezuela, Bolivarian Republic of': 'Venezuela'}
    ))

### Clean up labs

In [15]:
meta_working['submitting_lab'] = (
    meta_working['submitting_lab']
    .replace(
        {'Instituto Nacional de Investigacion en Salud Publica (INSPI)': \
         'Instituto Nacional de Investigación en Salud Pública INSPI',
         'Instituto de Salud Publica': \
         'Instituto de Salud Publica de Chile'}
    ))

In [16]:
meta_working['originating_lab'] = (
    meta_working['originating_lab']
    .replace(
        {'Instituto Nacional de Investigacion en Salud Publica (INSPI)': \
         'Instituto Nacional de Investigación en Salud Pública INSPI',
         'INSPI': \
         'Instituto Nacional de Investigación en Salud Pública INSPI',}
    ))

## Export for Augur
Export tab-delimited file (TSV) for input into Augur.

- Replace `NaN` values with a question mark character (`?`), as preferred by Augur.
- Drop raw columns that have been parsed to new, expanded columns.
- Put columns in preferred order. This is not necessary for Augur, but provides a standard order for my own human reading.
- **Drop duplicate strains.**

In [17]:
col_order = [
    'strain',
    'date',
    'virus',
    'region', 
    'country',
    'host', 
    'isolate_id',
    'submitting_lab',
    'originating_lab']

meta_working = meta_working[col_order]

meta_clean = meta_working.fillna('?')

meta_clean = meta_clean.drop_duplicates(subset='strain', keep=False)

meta_clean.to_csv(export_augur, sep='\t', index=False)

## List all sequences for including before filtering on criteria.

In [22]:
meta_clean['strain'].count()

256

In [20]:
for x in meta_clean['strain'].to_list():
    print(x)

A/Goose/Argentina/389-1/2023
A/goose/Argentina/SENASA-140223/2023
A/goose/Araucania/239189-1/2023
A/goose/Argentina/140223/2023
A/gull/Maule/SJCEIRR-2343291/2023
A/brown-hooded_gull/Los_Rios/SJCEIRR-2470941/2023
A/brown-hooded_gull/Los_Rios/SJCEIRR-2470931/2023
A/Belcher_gull/Peru/UNMSM-A267/2022
A/kelp_gull/Maule/239349/2023
A/gray_gull/Tarapaca/232825-1/2023
A/gull/Peru/LIM-INS-006/2023
A/Gull/Chile/7023-2/2022
A/Gull/Chile/7023-3/2022
A/black_skimmer/Chile/C61962/2022
A/black-necked_swan/Los_Rios/SJCEIRR-2472921/2023
A/black-necked_swan/Uruguay/UDELAR-014-M3/2023
A/black-necked_swan/Uruguay/UDELAR-078-M2/2023
A/turkey/Valparaiso/SJCEIRR-2455621/2023
A/Turkey/Argentina/1711-2/2023
A/Turkey/Argentina/1710-1/2023
A/Turkey/Argentina/1348-3/2023
A/Turkey/Argentina/753-1/2023
A/backyard_turkey/Uruguay/UDELAR-124-M6/2023
A/turkey/Araucania/241892-3/2023
A/turkey/Nuble/240489-1/2023
A/turkey/Nuble/241568-1/2023
A/tern/Espirito_Santo/MAPA-1339N2/2023
A/frigatebird/Rio_de_Janeiro/MAPA-1532N/2